In [26]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [27]:
#Path to the main directory
folder_path = '/content/gdrive/My Drive/Colab Notebooks/InnSure/pdl_query/'
profiles_path = folder_path + 'matchProfile_0419.pickle'
profileurls_path = folder_path + 'matchProfileUrl_0419.pickle'

In [28]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import pickle
import os
from ast import literal_eval

In [29]:
## loads a pickle/python object
## prints current time
def now(phrase):
    print(f"[{datetime.now()}]: {phrase}")
    
now("Success!")
def load(filename):
    now("starting")
    if (os.path.exists(filename)):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        now("ending")
        return data
    else:
        raise FileNotFoundError(f"could not find {filename}")


## saves as pickle object
def save(filename, obj):
    now("Starting")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
        print(f"Saved obj Object to {filename}!")
    now("Ending")
    return ''

[2022-04-19 19:56:07.155275]: Success!


In [31]:
profiles = load(profiles_path)

[2022-04-19 20:06:32.404343]: starting
[2022-04-19 20:06:32.910238]: ending


In [32]:
len(profiles)

1106

In [ ]:
profiles[0]

{'data': {'birth_date': '1972-01-31',
  'birth_year': 1972,
  'certifications': [],
  'countries': ['united states'],
  'education': [{'degrees': ['bachelors', 'bachelor of arts'],
    'end_date': '1994',
    'gpa': None,
    'majors': [],
    'minors': [],
    'school': {'type': 'post-secondary institution'},
    'start_date': '1990'},
   {'degrees': ['doctor of medicine', 'doctorates'],
    'end_date': '1998',
    'gpa': None,
    'majors': ['medicine'],
    'minors': [],
    'school': {'type': 'post-secondary institution'},
    'start_date': '1994'}],
  'emails': [{'address': 'adriel.kramer@midhosp.org', 'type': 'professional'},
   {'address': 'adrielkramer@yahoo.com', 'type': 'personal'},
   {'address': 'kadriel@yahoo.com', 'type': 'personal'},
   {'address': 'adriel.kramer@middlesexhospital.org',
    'type': 'current_professional'}],
  'experience': [{'breachJob': 0,
    'company': {'facebook_url': 'facebook.com/middlesexhospital',
     'founded': 1904,
     'id': 'middlesex-healt

## **Test flattening profile 1**

In [ ]:
profile1 = profiles[0] #dict

profile1['data'].keys()

#df = {}
#df['search_input_url'] = profile1['search_input_url']
#df['search_input_company'] = profile1['search_input_company']
#df['search_breach_year'] = profile1['search_breach_year']
profile1['data']['search_input_url']= profile1['search_input_url']
profile1['data']['search_input_company']=profile1['search_input_company']
profile1['data']['search_breach_year']=profile1['search_breach_year']

In [ ]:
del profile1['dataset_version']
del profile1['status']
del profile1['likelihood']

In [ ]:
profile1.keys()

dict_keys(['data', 'search_input_url', 'search_input_company', 'search_breach_year'])

In [ ]:
del profile1['search_input_url']
del profile1['search_input_company']
del profile1['search_breach_year']

In [ ]:
profile1.keys()
profile1= profile1['data']

In [ ]:
len(profiles)

1106

In [ ]:
prof = profile1.copy()

In [ ]:
# CLEAN STUFF I DONT NEED FROM EDUCATION:
clean = {}

for i in range(len(prof['education'])):
  edu = prof['education'][i]
  educationX = 'edu' + str(i)
  toAdd = ['degrees','end_date','start_date','majors']
  for i in toAdd:
    if i in list(edu):
      clean[educationX+'_'+i]= edu[i]
  for k in list(edu['school']):
    if k not in ['type']:
        del edu['school'][k]
    if k == 'type':
      clean[educationX+'_school'] =  edu['school'][k]

In [ ]:
#experience
for i in range(len(prof['experience'])):
  job = prof['experience'][i]
  jobX = 'job' + str(i)
  
  toAdd_exp = ['company','end_date','start_date','title','breachJob']
  toAdd_company = ['industry','name','size','type']
  toAdd_title = ['levels','name']
  for item1 in toAdd_exp:
    if item1 in list(job):
      if item1 == 'company':
        companyKeys = list(job[item1])
        for i in toAdd_company:
          if i in companyKeys:
            clean[jobX+'_'+i]= job[item1][i]
      elif item1 == 'title':
        titleKeys = list(job[item1])
        for j in toAdd_title:
          if j in titleKeys:
            clean[jobX+'_'+j]= job[item1][j]
      else:
        clean[jobX+'_'+item1]= job[item1]

In [ ]:
#other stuff
addYN = ['linkedin_url','facebook_url','twitter_url','github_url','mobile_phone','phone_numbers','work_email','personal_emails']
addLen = ['emails','profiles','street_addresses','location_names']
addOthers = ['id','full_name','first_name','last_name','gender','birth_year','linkedin_connections','inferred_salary','interests','skills','certifications','languages']

for i in addYN:
  if i in list(prof):
    if prof[i] != None:
      clean[i] = 1
    else:
      clean[i] = 0

for i in list(addLen):
  if i in list(prof):
    clean[i] = len(prof[i])

for i in list(addOthers):
  if i in list(prof):
    clean[i] = prof[i]

In [ ]:
for k in list(clean):
  if clean[k] == None:
    clean[k] = ""
  else:
    clean[k] = str(clean[k])


In [ ]:
clean
#df_clean = pd.DataFrame.from_dict(clean)
df_clean = pd.DataFrame(clean,index=[0,])
#df

In [ ]:
df_clean.info()

In [ ]:
profs = profiles[:5]
for profile in profs:
  print(profile)

{'data': {'id': '159wWWs7bM6hRXD1l1qmLQ_0000', 'full_name': 'adriel kramer', 'first_name': 'adriel', 'middle_initial': 'a', 'middle_name': None, 'last_initial': 'k', 'last_name': 'kramer', 'gender': None, 'birth_year': 1972, 'birth_date': '1972-01-31', 'linkedin_url': 'linkedin.com/in/adriel-kramer-4b466a41', 'linkedin_username': 'adriel-kramer-4b466a41', 'linkedin_id': '147948616', 'facebook_url': None, 'facebook_username': None, 'facebook_id': None, 'twitter_url': None, 'twitter_username': None, 'github_url': None, 'github_username': None, 'work_email': 'adriel.kramer@middlesexhospital.org', 'personal_emails': ['adrielkramer@yahoo.com', 'kadriel@yahoo.com'], 'mobile_phone': None, 'industry': 'hospital & health care', 'job_title': 'chief medical information officer', 'job_title_role': 'health', 'job_title_sub_role': None, 'job_title_levels': ['cxo'], 'job_onet_major_group': 'management occupations', 'job_onet_minor_group': 'top executives', 'job_onet_broad_occupation': 'chief executiv

In [33]:
################ DO THIS ON 1106 PROFILES ####################

addYN = ['linkedin_url','facebook_url','twitter_url','github_url','mobile_phone','phone_numbers','work_email','personal_emails']
addLen = ['emails','profiles','street_addresses','location_names']
addOthers = ['id','full_name','first_name','last_name','gender','birth_year','linkedin_connections','inferred_salary','interests','skills','certifications','languages']

profs = profiles[:5]

new_profs = []
for profile in profs:
  profile['data'].keys()
  print(profile['search_input_url'])
  profile['data']['search_input_url']= profile['search_input_url']
  profile['data']['search_input_company']=profile['search_input_company']
  profile['data']['search_breach_year']=profile['search_breach_year']
  del profile['dataset_version']
  del profile['status']
  del profile['likelihood']
  del profile['search_input_url']
  del profile['search_input_company']
  del profile['search_breach_year']
  profile= profile['data']
  
  cleanProfile = {}

  ### EDUCATION
  for i in range(len(profile['education'])):
    edu = profile['education'][i]
    educationX = 'edu' + str(i)
    toAdd = ['degrees','end_date','start_date','majors']
    for i in toAdd:
      if i in list(edu):
        cleanProfile[educationX+'_'+i]= edu[i]
    for k in list(edu['school']):
      if k not in ['type']:
          del edu['school'][k]
      if k == 'type':
        cleanProfile[educationX+'_school'] =  edu['school'][k]

    ### EXPERIENCE
    for i in range(len(profile['experience'])):
      job = profile['experience'][i]
      jobX = 'job' + str(i)
      
      toAdd_exp = ['company','end_date','start_date','title','breachJob']
      toAdd_company = ['industry','name','size','type']
      toAdd_title = ['levels','name']
      for item1 in toAdd_exp:
        if item1 in list(job):
          if item1 == 'company':
            companyKeys = list(job[item1])
            for i in toAdd_company:
              if i in companyKeys:
                cleanProfile[jobX+'_'+i]= job[item1][i]
          elif item1 == 'title':
            titleKeys = list(job[item1])
            for j in toAdd_title:
              if j in titleKeys:
                cleanProfile[jobX+'_'+j]= job[item1][j]
          else:
            cleanProfile[jobX+'_'+item1]= job[item1]
    ### 
    for i in addYN:
      if i in list(profile):
        if profile[i] != None:
          cleanProfile[i] = 1
        else:
          cleanProfile[i] = 0

    for i in list(addLen):
      if i in list(profile):
        cleanProfile[i] = len(profile[i])

    for i in list(addOthers):
      if i in list(profile):
        cleanProfile[i] = profile[i]

  for k in list(cleanProfile):
    if cleanProfile[k] == None:
      cleanProfile[k] = ""
    else:
      cleanProfile[k] = str(cleanProfile[k])
  new_profs.append(cleanProfile)


https://www.linkedin.com/in/adriel-kramer-4b466a41
https://www.linkedin.com/in/ajirapatnakul
https://www.linkedin.com/in/amanda-hammel
https://www.linkedin.com/in/andrew-rosenberg-md-62902527
https://www.linkedin.com/in/angelalambmd


In [ ]:
df_new_profs = pd.DataFrame(new_profs)

In [ ]:
list(df_new_profs.columns)

['edu0_degrees',
 'edu0_end_date',
 'edu0_start_date',
 'edu0_majors',
 'edu0_school',
 'job0_industry',
 'job0_name',
 'job0_size',
 'job0_type',
 'job0_end_date',
 'job0_start_date',
 'job0_levels',
 'job0_breachJob',
 'job1_industry',
 'job1_name',
 'job1_size',
 'job1_type',
 'job1_end_date',
 'job1_start_date',
 'job1_levels',
 'job1_breachJob',
 'linkedin_url',
 'facebook_url',
 'twitter_url',
 'github_url',
 'mobile_phone',
 'phone_numbers',
 'work_email',
 'personal_emails',
 'emails',
 'profiles',
 'street_addresses',
 'location_names',
 'id',
 'full_name',
 'first_name',
 'last_name',
 'gender',
 'birth_year',
 'linkedin_connections',
 'inferred_salary',
 'interests',
 'skills',
 'certifications',
 'languages',
 'edu1_degrees',
 'edu1_end_date',
 'edu1_start_date',
 'edu1_majors',
 'edu1_school',
 'job2_industry',
 'job2_name',
 'job2_size',
 'job2_type',
 'job2_end_date',
 'job2_start_date',
 'job2_levels',
 'job2_breachJob',
 'job3_industry',
 'job3_name',
 'job3_size',
 'j

In [ ]:
df_new_profs['edu5_majors'].tolist()

[nan, nan, nan, "['health care', 'informatics']", nan]

In [35]:
profiles[1]

{'data': {'birth_date': None,
  'birth_year': None,
  'certifications': [{'end_date': None,
    'name': 'license q9k3e7wtgt',
    'organization': 'coursera verified certificates',
    'start_date': '2014-05'},
   {'end_date': None,
    'name': 'license jg2xcv6bzs',
    'organization': 'coursera verified certificates',
    'start_date': '2014-06'},
   {'end_date': None,
    'name': 'exploratory data analysis',
    'organization': 'coursera',
    'start_date': '2014-08'},
   {'end_date': None,
    'name': 'getting and cleaning data',
    'organization': 'coursera',
    'start_date': '2014-09'},
   {'end_date': None,
    'name': 'reproducible research',
    'organization': 'coursera',
    'start_date': '2014-12'},
   {'end_date': None,
    'name': 'the data scientist’s toolbox',
    'organization': None,
    'start_date': None},
   {'end_date': None,
    'name': 'r programming',
    'organization': None,
    'start_date': None},
   {'end_date': None,
    'name': 'coursera verified certifi